In [ ]:
# Limpar todas as variáveis
from IPython import get_ipython
get_ipython().magic('reset -sf')

# Código para reiniciar o ambiente do Colab
import os
os.kill(os.getpid(), 9)

In [ ]:
# Importando as bibliotecas necessárias
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta
import gc

# Liberar memória manualmente
_ = gc.collect()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
def preenche_falhas2(dados, codigo, inicio, fim, freq="15min"):

    # Gerar todas as datas no intervalo de 15 minutos
    datas_completas = pd.date_range(start = inicio, end = fim, freq='15min')

    # Criar lista para armazenar os dados completos
    dados_completos = []

    # Iterar sobre as datas completas
    for data_hora in datas_completas:

      # Verificar se a data está no dataframe original
        linha = df[df['DataHora'] == data_hora]

        if not linha.empty:
          # Se existir, adicionar a linha ao novo dataframe
          dados_completos.append(linha.iloc[0].tolist())

        else:
          # Caso contrário, adicionar os valores faltantes
          dados_completos.append([data_hora, 58730001, -999, -999])

    return dados_completos

In [ ]:
# funcao para indicar as datas de falha nos dados de telemetria
def preenche_falhas(dataframe, cod, inicio, fim, freq="15min"):
    """
    Gera uma sequência de data e hora com base no intervalo fornecido.

    Parâmetros:
        inicio (str): Data e hora inicial (formato: 'YYYY-MM-DD HH:MM:SS').
        fim (str): Data e hora final (formato: 'YYYY-MM-DD HH:MM:SS').
        freq (str): Frequência dos intervalos (exemplo: '15T' para 15 minutos).

    Retorna:
        pandas.DataFrame: DataFrame contendo a sequência de datas e horas.
    """
    # converte a data em string para formato datetime
    # inicio = datetime.strptime(inicio, "%d/%m/%Y")
    # fim = datetime.strptime(fim, "%d/%m/%Y")

    # cria a sequencia completa de passos temporais
    datas = pd.date_range(start=inicio, end=fim, freq=freq)

    # cria pandas dataframe com uma coluna
    datas = pd.DataFrame(datas,columns=["DataHora"])

    # Convertendo os dados de entrada para um DataFrame do pandas
    dataframe = pd.DataFrame(dataframe)

    # converte coluna DataHora para o formato datetime
    dataframe['DataHora'] = pd.to_datetime(dataframe['DataHora'])

    # mesclar o dataframe completo de dados com o original, mantendo linhas puladas vazias

    # preenchendo a linha vazia
    pd.merge(datas, dataframe, on='DataHora', how='left')['CodEstacao'] = pd.merge(datas, dataframe, on='DataHora', how='left')['CodEstacao'].fillna(cod) # coluna do codigo
    pd.merge(datas, dataframe, on='DataHora', how='left')['Vazao'] = pd.merge(datas, dataframe, on='DataHora', how='left')['Vazao'].fillna(-999) # troca NA por -999
    pd.merge(datas, dataframe, on='DataHora', how='left')['Nivel'] = pd.merge(datas, dataframe, on='DataHora', how='left')['Nivel'].fillna(-999) # troca NA por -999
    pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'] = pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'].fillna(-999) # troca NA por -999
    pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'] = (pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'].fillna(-999).astype(float))  # Ou .astype(int), dependendo do tipo desejado

    return pd.merge(datas, dataframe, on='DataHora', how='left')

In [19]:
caminho_drive = "/content/drive/MyDrive/2024-2025"

Inserir a data para inicio da previsao

In [21]:
inicio_previsao = "22/12/2024 09:00:00"

# Converter para objeto de data
inicio_previsao = datetime.strptime(inicio_previsao, "%d/%m/%Y %H:%M:%S")

# converter no formato US
final = inicio_previsao.strftime("%Y-%m-%d")

# data final do intervalo de dados
inicio_serie = inicio_previsao - timedelta(days=5)

# converter para o formato US
inicio = inicio_serie.strftime("%Y-%m-%d")

# converte para o formato BR
data_inicio_serie = inicio_serie.strftime("%d/%m/%Y")
data_final_serie  = inicio_previsao.strftime("%d/%m/%Y")

# codigos das estacoes de interesse
codigos_estacao  = ['58730001','58735000','58736000','58765010','58770000','58790002','58792100',
                    '58880001','58920000','58930000','58934000','58940000','58960000','58974000']

In [22]:
# URL do serviço
url = "https://telemetriaws1.ana.gov.br/ServiceANA.asmx/DadosHidrometeorologicos"

In [23]:
# Criando um dicionário para armazenar os DataFrames
dados_estacoes = {}

In [29]:
codigo = codigos_estacao[1]
codigo

params = {
        "CodEstacao": codigo,
        "DataInicio": data_inicio_serie,
        "DataFim": data_final_serie
    }

# Realizando a requisição GET
response = requests.get(url, params=params)

# Verificando o status da resposta
if response.status_code != 200:
  print(f"Erro na requisição para o código {codigo}: {response.status_code}")


# Parsing do conteúdo XML
xml_content = response.content
root = ET.fromstring(xml_content)

    # Extraindo os dados
dados = []

for entry in root.findall(".//DocumentElement/DadosHidrometereologicos"):
  dados.append({
      "CodEstacao": entry.find("CodEstacao").text,
      "DataHora": entry.find("DataHora").text,
      "Vazao": entry.find("Vazao").text,
      "Nivel": entry.find("Nivel").text,
      "Chuva": entry.find("Chuva").text,
      })

In [ ]:
# gera sequencia e preenche periodo
dados_completos = preenche_falhas(dataframe=dados,cod = codigo, inicio=inicio_serie, fim=inicio_previsao, freq="15min")


In [8]:
for codigo in codigos_estacao:

    print(f"Buscando dados para o código de estação: {codigo}")
    # Parâmetros da requisição
    params = {
        "CodEstacao": codigo,
        "DataInicio": data_inicio_serie,
        "DataFim": data_final_serie
    }

    # Realizando a requisição GET
    response = requests.get(url, params=params)

    # Verificando o status da resposta
    if response.status_code != 200:
        print(f"Erro na requisição para o código {codigo}: {response.status_code}")
        continue

    # Parsing do conteúdo XML
    xml_content = response.content
    root = ET.fromstring(xml_content)

    # Extraindo os dados
    dados = []
    for entry in root.findall(".//DocumentElement/DadosHidrometereologicos"):
        dados.append({
            "CodEstacao": entry.find("CodEstacao").text,
            "DataHora": entry.find("DataHora").text,
            "Vazao": entry.find("Vazao").text,
            "Nivel": entry.find("Nivel").text,
            "Chuva": entry.find("Chuva").text,
        })

    # gera sequencia e preenche periodo
    dados_completos = preenche_falhas(dataframe=dados,cod = codigo, inicio=inicio_serie, fim=inicio_previsao, freq="15min")

    # Salvando o DataFrame no dicionário
    dados_estacoes[codigo] = dados_completos

# Salvando os dados em um único arquivo Excel, com uma aba para cada estação
with pd.ExcelWriter(f"{caminho_drive}/dados_hidrometeorologicos_{inicio}_{final}.xlsx") as writer:
    for codigo, df in dados_estacoes.items():
        df.to_excel(writer, sheet_name=f"Estacao_{codigo}", index=False)

print("Dados salvos no arquivo 'dados_hidrometeorologicos.xlsx'")

Buscando dados para o código de estação: 58730001
Buscando dados para o código de estação: 58735000
Buscando dados para o código de estação: 58736000
Buscando dados para o código de estação: 58765010
Buscando dados para o código de estação: 58770000
Buscando dados para o código de estação: 58790002
Buscando dados para o código de estação: 58792100
Buscando dados para o código de estação: 58880001
Buscando dados para o código de estação: 58920000
Buscando dados para o código de estação: 58930000
Buscando dados para o código de estação: 58934000
Buscando dados para o código de estação: 58940000
Buscando dados para o código de estação: 58960000


<ipython-input-3-e0db21afce57>:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'] = pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'].fillna(-999) # troca NA por -999
<ipython-input-3-e0db21afce57>:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'] = (pd.merge(datas, dataframe, on='DataHora', how='left')['Chuva'].fillna(-999).astype(float))  # Ou .astype(int), dependendo do tipo desejado


Buscando dados para o código de estação: 58974000
Dados salvos no arquivo 'dados_hidrometeorologicos.xlsx'
